In [1]:
import pandas as pd

In [2]:
#Read socioeconomic status per zipcode

In [3]:
socioeconomic_df = pd.read_csv("acs_socioeconomic_2019.csv")

In [4]:
#Read causal effect file(georgia)

In [5]:
ga_treatment_effect = pd.read_csv('GA_climate_causal_effect.csv')

In [6]:
ga_treatment_effect.head()

,zip,1,2,3,4,5,6,elderlyReturns,returnsTotalwSalariesWages,returnsDependentCareCredit,returnsEducationCredit,risk_group,propensity_score,price_shift,y_0,y_0_corrected,y_1,y_1_corrected,treatment_effect,treatment_effect_no_corr
0,29803,0.242633,0.193517,0.138998,0.103635,0.235756,0.085462,0.423870,0.716601,0.210216,0.040766,False,0.366196,6.432405,-0.508141,10.442471,11.188678,11.188678,0.746207,11.696819
1,29829,0.265079,0.233333,0.163492,0.123810,0.190476,0.023810,0.195238,0.865079,0.322222,0.061905,False,0.282418,19.079154,14.949684,20.704386,9.500285,9.500285,-11.204101,-5.449398
2,29841,0.286785,0.242507,0.162807,0.105586,0.155995,0.046322,0.266349,0.816757,0.247956,0.053815,True,0.310067,13.240693,11.546645,11.546645,11.542999,17.018242,5.471597,-0.003646
3,30004,0.206252,0.126652,0.102159,0.085079,0.209152,0.270706,0.215598,0.834354,0.298421,0.040928,False,0.311576,5.631119,3.788156,6.465231,1.159813,1.159813,-5.305419,-2.628344
4,30005,0.202146,0.118012,0.093168,0.081310,0.243930,0.261434,0.208922,0.865613,0.341615,0.053077,False,0.303279,3.725923,4.785729,3.264595,-2.438791,-2.438791,-5.703386,-7.224520


In [7]:
#Merge by zip

In [8]:
socioeconomic_df['zip'] = socioeconomic_df['zip'].astype(str).str.zfill(5)
ga_treatment_effect['zip'] = ga_treatment_effect['zip'].astype(str).str.zfill(5)

In [9]:
ga_socioeconomic_treatment_effect = pd.merge(socioeconomic_df, ga_treatment_effect, on="zip")

In [10]:
#OLS regression treatment effect ~ socioeconomic variables

In [11]:
import statsmodels.api as sm

X = ga_socioeconomic_treatment_effect[['pct_poverty', 'pct_black', 'pct_college_plus', 'pct_mortgage']]
X = sm.add_constant(X)
y = ga_socioeconomic_treatment_effect['treatment_effect_no_corr']

model = sm.OLS(y, X).fit()
print(model.summary())

                               OLS Regression Results                               
Dep. Variable:     treatment_effect_no_corr   R-squared:                       0.592
Model:                                  OLS   Adj. R-squared:                  0.585
Method:                       Least Squares   F-statistic:                     83.53
Date:                      Thu, 08 May 2025   Prob (F-statistic):           1.08e-43
Time:                              20:42:16   Log-Likelihood:                -713.78
No. Observations:                       235   AIC:                             1438.
Df Residuals:                           230   BIC:                             1455.
Df Model:                                 4                                         
Covariance Type:                  nonrobust                                         
                       coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------

In [12]:
# Mean treatment effect

In [13]:
ga_socioeconomic_treatment_effect['treatment_effect_no_corr'].mean()

-4.365897140699997

In [14]:
# Mean treatment effect: Lower 25% & Upper 25%

In [15]:
low_poverty = ga_socioeconomic_treatment_effect[ga_socioeconomic_treatment_effect['pct_poverty'] <= ga_socioeconomic_treatment_effect['pct_poverty'].quantile(0.25)]

In [16]:
low_poverty['treatment_effect_no_corr'].mean()

-0.43794904792995015

In [17]:
high_poverty = ga_socioeconomic_treatment_effect[ga_socioeconomic_treatment_effect['pct_poverty'] >= ga_socioeconomic_treatment_effect['pct_poverty'].quantile(0.75)]

In [18]:
high_poverty['treatment_effect_no_corr'].mean()

-10.578714177651669

In [19]:
low_black = ga_socioeconomic_treatment_effect[ga_socioeconomic_treatment_effect['pct_black'] <= ga_socioeconomic_treatment_effect['pct_black'].quantile(0.25)]

In [20]:
low_black['treatment_effect_no_corr'].mean()

2.481723415275688

In [21]:
high_black = ga_socioeconomic_treatment_effect[ga_socioeconomic_treatment_effect['pct_black'] >= ga_socioeconomic_treatment_effect['pct_black'].quantile(0.75)]

In [22]:
high_black['treatment_effect_no_corr'].mean()

-11.343868659147251

In [23]:
low_college = ga_socioeconomic_treatment_effect[ga_socioeconomic_treatment_effect['pct_college_plus'] <= ga_socioeconomic_treatment_effect['pct_college_plus'].quantile(0.25)]

In [24]:
low_college['treatment_effect_no_corr'].mean()

-5.027099713150243

In [25]:
high_college = ga_socioeconomic_treatment_effect[ga_socioeconomic_treatment_effect['pct_college_plus'] >= ga_socioeconomic_treatment_effect['pct_college_plus'].quantile(0.75)]

In [26]:
high_college['treatment_effect_no_corr'].mean()

-1.8009960239909886

In [27]:
low_mortgage = ga_socioeconomic_treatment_effect[ga_socioeconomic_treatment_effect['pct_mortgage'] <= ga_socioeconomic_treatment_effect['pct_mortgage'].quantile(0.25)]

In [28]:
low_mortgage['treatment_effect_no_corr'].mean()

-2.5798692033962745

In [29]:
high_mortgage = ga_socioeconomic_treatment_effect[ga_socioeconomic_treatment_effect['pct_mortgage'] >= ga_socioeconomic_treatment_effect['pct_mortgage'].quantile(0.75)]

In [30]:
high_mortgage['treatment_effect_no_corr'].mean()

-6.622256574424799

In [31]:
#Get Georgia socioeconomic status

In [32]:
socioeconomic_df = pd.read_csv("acs_socioeconomic_2019.csv")

In [33]:
zipdata = pd.read_excel("uszips.xlsx")

In [34]:
socioeconomic_df['zip'] = socioeconomic_df['zip'].astype(str).str.zfill(5)
zipdata['zip'] = zipdata['zip'].astype(str).str.zfill(5)

In [35]:
georgia_zipdata = zipdata[zipdata['state_name'] == 'Georgia']

In [36]:
ga_socioeconomic = pd.merge(socioeconomic_df, georgia_zipdata, on="zip")

In [37]:
ga_socioeconomic = ga_socioeconomic.dropna(subset=['pct_poverty', 'pct_black', 'pct_college_plus', 'pct_mortgage'])

In [38]:
#Cluster zip using socioeconomic variables

In [39]:
from sklearn.preprocessing import StandardScaler

In [40]:
features = ['pct_poverty', 'pct_black', 'pct_college_plus', 'pct_mortgage']

In [41]:
scaler = StandardScaler()
ga_socioeconomic_scaled = scaler.fit_transform(ga_socioeconomic[features])

In [42]:
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score

In [43]:
#Clustering
range_n_clusters = [2, 3, 4, 5, 6, 7, 8, 9, 10]
for k in range_n_clusters: #try different numbers of clusters
    kmeans = KMeans(n_clusters=k, random_state=324)
    cluster_labels = kmeans.fit_predict(ga_socioeconomic_scaled)
    #report average Silhouette score
    silhouette_avg = silhouette_score(ga_socioeconomic_scaled, cluster_labels)
    print("For n_clusters ={},".format(k)+" the average silhouette_score is :{}".format(silhouette_avg))

C:\Users\user\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


For n_clusters =2, the average silhouette_score is :0.2901922419337133
For n_clusters =3, the average silhouette_score is :0.29582125849179336
For n_clusters =4, the average silhouette_score is :0.2684963884737789


C:\Users\user\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
C:\Users\user\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
C:\Users\user\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


For n_clusters =5, the average silhouette_score is :0.264160633333385
For n_clusters =6, the average silhouette_score is :0.2619565624785483
For n_clusters =7, the average silhouette_score is :0.2436276772897536


C:\Users\user\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
C:\Users\user\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
C:\Users\user\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


For n_clusters =8, the average silhouette_score is :0.24470327613666734
For n_clusters =9, the average silhouette_score is :0.2262880482951657
For n_clusters =10, the average silhouette_score is :0.23786007803821052


C:\Users\user\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
C:\Users\user\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


In [44]:
kmeans = KMeans(n_clusters=3, random_state=324)
cluster_labels = kmeans.fit_predict(ga_socioeconomic_scaled)

C:\Users\user\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


In [45]:
ga_socioeconomic['cluster'] = cluster_labels

In [46]:
cluster_profile = ga_socioeconomic.groupby('cluster')[features].mean()
print(cluster_profile)

         pct_poverty  pct_black  pct_college_plus  pct_mortgage
cluster                                                        
0           0.272380   0.508756          0.139242      0.450277
1           0.153791   0.130559          0.163720      0.493802
2           0.096582   0.256235          0.445186      0.721921


In [47]:
ga_socioeconomic

,zip,pct_poverty,pct_black,pct_college_plus,pct_mortgage,lat,lng,city,state_id,state_name,...,density,county_fips,county_name,county_weights,county_names_all,county_fips_all,imprecise,military,timezone,cluster
0,30663,0.218740,0.161598,0.161290,0.511719,33.61572,-83.60357,Rutledge,GA,Georgia,...,26.9,13211,Morgan,"{""13211"": 100}",Morgan,13211,False,False,America/New_York,1
1,30666,0.080728,0.149871,0.256974,0.704267,33.95891,-83.58266,Statham,GA,Georgia,...,114.7,13013,Barrow,"{""13013"": 59.72, ""13219"": 30.58, ""13157"": 9.7}",Barrow|Oconee|Jackson,13013|13219|13157,False,False,America/New_York,2
2,30115,0.038252,0.038221,0.405660,0.739922,34.21296,-84.40138,Canton,GA,Georgia,...,281.2,13057,Cherokee,"{""13057"": 100}",Cherokee,13057,False,False,America/New_York,2
3,30117,0.264576,0.234278,0.256777,0.573240,33.58113,-85.13197,Carrollton,GA,Georgia,...,138.9,13045,Carroll,"{""13045"": 100}",Carroll,13045,False,False,America/New_York,1
4,30127,0.066633,0.432189,0.372503,0.761084,33.87647,-84.69648,Powder Springs,GA,Georgia,...,542.6,13067,Cobb,"{""13067"": 84.55, ""13223"": 15.45}",Cobb|Paulding,13067|13223,False,False,America/New_York,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
724,31518,0.251876,0.050375,0.236542,0.397541,31.51272,-82.18325,Bristol,GA,Georgia,...,6.8,13001,Appling,"{""13001"": 57.48, ""13229"": 30.81, ""13305"": 11.71}",Appling|Pierce|Wayne,13001|13229|13305,False,False,America/New_York,1
725,31543,0.198361,0.028996,0.057479,0.491042,31.34455,-81.82697,Hortense,GA,Georgia,...,11.5,13025,Brantley,"{""13025"": 50.37, ""13305"": 41.61, ""13127"": 8.02}",Brantley|Wayne|Glynn,13025|13305|13127,False,False,America/New_York,1
726,31632,0.105569,0.087518,0.300072,0.659652,30.98760,-83.36006,Hahira,GA,Georgia,...,52.6,13185,Lowndes,"{""13185"": 78.93, ""13075"": 21.07}",Lowndes|Cook,13185|13075,False,False,America/New_York,2
727,31018,0.181734,0.573139,0.058635,0.377358,32.97888,-82.62012,Davisboro,GA,Georgia,...,11.2,13303,Washington,"{""13303"": 100}",Washington,13303,False,False,America/New_York,0


In [48]:
ga_socioeconomic.to_csv("ga_socioeconomic.csv", index=False)